In [4]:
import json
import ast
import pandas as pd
from collections import defaultdict
import time

In [5]:
def businessSummary(business_id):
#     business_dict = defaultdict(dict) 
    for line in open("yelp_dataset/business.json", "r"): 
        curline = json.loads(line)
        if curline['business_id'] == business_id:
            if curline['attributes'] is not None:
                attributes = {}
                for j in curline['attributes'].keys():
                    attributes[j] = ast.literal_eval(curline['attributes'][j])
                attributes = pd.io.json.json_normalize(attributes, sep='_').to_dict(orient='records')[0]
            else:
                attributes = {}
            if curline['categories'] is not None:
                categories = {'categories': curline['categories'].split(', ')}
            else:
                categories = {'categories': []}
#             categories = flattenfurther(categories)
            categories = pd.io.json.json_normalize(categories, sep='_').to_dict(orient='records')[0]
            attributes.update(categories)
#             business_dict[curline['business_id']] = attributes
            #return attributes
    newx = {}
    for key in list(attributes.keys()):
        if key != 'categories':
            try:
                valeval = ast.literal_eval(attributes[key]) 
                newx[key] = valeval
    #             print("try")
            except:
    #             print("except")
                newx[str(key) + '_' + str(attributes[key])] = True
        else:
            for cat in attributes[key]:
                newx[str(key) + '_' + str(cat)] = True
    return newx

In [2]:
businessSummary('u2q_84hHvKGl5hKnAE7zNw')
rel
for i in range(0,100):
    
    np.asarray(list(newran().values()))

SyntaxError: invalid syntax (<ipython-input-2-064420497aee>, line 8)

In [3]:
lrcoefs , _ , _ = _logistic_regression_path(X = vecs,y = labels,fit_intercept = False,solver = 'newton-cg',multi_class = 'ovr',max_iter = 1000)

NameError: name '_logistic_regression_path' is not defined